In [17]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [18]:
from openai import OpenAI

api_key = "sk-XpXS3mBjka5gX86ZmE6YveGAZeSLkPJjNvJKd9AF23rF17P0"
api_base = "https://api.zetatechs.com/v1"
client = OpenAI(api_key=api_key, base_url=api_base)

# completion = client.chat.completions.create(
#   model="gpt-4o-mini",
#   messages=[
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "Hello!"}
#   ]
# )

# print(completion.choices[0].message)

In [19]:
from datasets import load_dataset

# 加载维基百科数据集
wiki_dataset = load_dataset(
    'wikimedia/wikipedia', 
    '20231101.zh',
)

print(wiki_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 1384748
    })
})


In [20]:
# print(type(wiki_dataset['train'][0]), type(wiki_dataset['train']), wiki_dataset['train'][0].keys())
# print(type(wiki_dataset['train']), type(wiki_dataset['train']['text']))
# print(len(wiki_dataset['train']['text']))

In [21]:
# claude 给出的处理方式

In [22]:
from openai import OpenAI
import json
from tqdm import tqdm
import time
from datasets import load_dataset
import random

In [23]:
def process_wiki_with_gpt(text, task_type): 
    """
    使用GPT处理维基百科文本
    text 是wiki_dataset['train']['text'] 的元素。
    task_type 是 'summary', 'qa', 'concept' 之一
    """
    
    # 针对不同任务类型的提示模板
    # 过往版本储存在: "基础阶段_早期的prompts.txt"
    # 版本七
    prompts = {
        "summary": """请对以下维基百科文章进行总结。
        你的输出必须是一个JSON格式，其中input字段必须是完整的原始文章内容，output字段是你的总结。
        格式示例：
        {"instruction": "请总结以下内容：", "input": "<这里要填入完整的原始文章>", "output": "<这里是总结内容，使用普通文本格式>"}

        待总结的文章内容：""",
                    
        "qa": """请基于以下维基百科文章生成1个高质量的问答对。
        你的输出必须是一个JSON格式，必须严格遵循以下规则：
        1. input字段可以包含背景信息和问题，或仅包含问题，格式可以是：
        "背景：<相关背景信息>\n\n问题：<具体问题>"
        或
        "问题：<具体问题>"
        或
        "<具体问题>"
        等
        2. output字段仅包含问题的答案
        3. instruction字段固定为"请回答以下问题："

        格式示例：
        {
            "instruction": "请回答以下问题：",
            "input": "背景：2008年奥运会在北京举办。\n\n问题：2008年北京奥运会的口号是什么？",
            "output": "2008年北京奥运会的口号是'同一个世界，同一个梦想'。"
        }
        或
        {
            "instruction": "请回答以下问题：",
            "input": "问题：2008年北京奥运会的口号是什么？",
            "output": "2008年北京奥运会的口号是'同一个世界，同一个梦想'。"
        }
        或
        {
            "instruction": "请回答以下问题：",
            "input": "2008年北京奥运会的口号是什么？",
            "output": "2008年北京奥运会的口号是'同一个世界，同一个梦想'。"
        }

        文章内容：""",
                    
        "concept": """请从以下维基百科文章中提取一个关键概念并解释。
        你的输出必须是一个JSON格式，必须严格遵循以下规则：
        1. 从文章中识别最重要的一个概念或专有名词
        2. input字段可以包含领域和概念名称，或仅包含概念名称，格式可以是：
        "在<领域>中，<概念名称>（指代范围或定义）需要解释。"
        或
        "请解释<概念名称>。"
        等
        3. output字段必须包含该概念的详细解释，包括定义、特点、历史背景或应用等方面
        4. instruction字段固定为"请解释下列概念："

        格式示例：
        {
            "instruction": "请解释下列概念：",
            "input": "在计算机科学中，递归（作为一种编程技术和算法策略）需要解释。",
            "output": "递归是一种解决问题的方法，通过函数或算法调用自身来处理问题的更小实例。它具有以下特点：1. 基本情况（终止条件）；2. 递归情况（将问题分解为更小的子问题）；3. 调用栈的使用。递归在处理树结构、图遍历、分治算法等场景中广泛应用，是计算机科学中的基础概念。"
        }
        或
        {
            "instruction": "请解释下列概念：",
            "input": "请解释什么是递归",
            "output": "递归是一种解决问题的方法，通过函数或算法调用自身来处理问题的更小实例。它具有以下特点：1. 基本情况（终止条件）；2. 递归情况（将问题分解为更小的子问题）；3. 调用栈的使用。递归在处理树结构、图遍历、分治算法等场景中广泛应用，是计算机科学中的基础概念。"
        }
        或{
            "instruction": "请解释下列概念：",
            "input": "什么是递归?",
            "output": "递归是一种解决问题的方法，通过函数或算法调用自身来处理问题的更小实例。它具有以下特点：1. 基本情况（终止条件）；2. 递归情况（将问题分解为更小的子问题）；3. 调用栈的使用。递归在处理树结构、图遍历、分治算法等场景中广泛应用，是计算机科学中的基础概念。"
        }

        文章内容："""
    }


    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # 使用GPT-4以获得最好的效果
            messages=[
                {"role": "system", "content": "你是一个专业的数据处理助手。请确保所有输出都是有效的JSON格式，使用双引号作为字符串定界符。"},
                {"role": "user", "content": f"{prompts[task_type]} {text}"}
            ]
        )
        
        # 添加错误处理和清理
        content = response.choices[0].message.content.strip()
        try:
            result = json.loads(content)
            result['type'] = f'wiki_{task_type}'
            return result
        except json.JSONDecodeError as e:
            # 记录错误信息到文件
            with open('error_log.txt', 'a', encoding='utf-8') as f:
                f.write('\n' + '='*50 + '\n')  # 分隔符
                f.write(f'时间: {time.strftime("%Y-%m-%d %H:%M:%S")}\n')
                f.write(f'任务类型: {task_type}\n')
                f.write(f'错误类型: JSON解析错误\n')
                f.write(f'错误信息: {str(e)}\n')
                f.write(f'原始文本: {text[:200]}...\n')  # 只记录前200个字符
                f.write(f'API响应: {content}\n')
                f.write('='*50 + '\n')
            print(f"JSON解析错误: {e}")
            return None
    
    except Exception as e:
        # 记录其他类型的错误
        with open('error_log.txt', 'a', encoding='utf-8') as f:
            f.write('\n' + '='*50 + '\n')  # 分隔符
            f.write(f'时间: {time.strftime("%Y-%m-%d %H:%M:%S")}\n')
            f.write(f'任务类型: {task_type}\n')
            f.write(f'错误类型: {type(e).__name__}\n')
            f.write(f'错误信息: {str(e)}\n')
            f.write(f'原始文本: {text[:200]}...\n')  # 只记录前200个字符
            f.write('='*50 + '\n')
        print(f"处理出错: {e}")
        return None

def main():
    # 加载维基百科数据集
    wiki_dataset = load_dataset(
        'wikimedia/wikipedia', 
        '20231101.zh',
    )
    
    # 初始化结果列表
    processed_data = []
    
    # 设置处理的样本数量
    sample_size = 50000 + 1000  # 可以根据需要调整，适当增加，因为可能会处理失败
    
    # 随机采样数据
    selected_texts = random.sample(wiki_dataset['train']['text'], sample_size) # 从 wiki_dataset['train']['text'] 中随机采样 sample_size 条数据
    
    # 任务类型列表
    task_types = ['summary', 'qa', 'concept']
    
    # 处理数据
    for text in tqdm(selected_texts, desc="处理维基百科数据"): # selected_texts 是一个 list
        if len(text) < 100:  # 跳过过短的文本
            continue
            
        # 随机选择一个任务类型
        task_type = random.choice(task_types)
        
        # 使用GPT处理文本
        result = process_wiki_with_gpt(text[:2000], task_type)  # 限制输入长度
        
        if result: # 如果 result 不为 None，也即是处理成功
            processed_data.append(result) # result 是一个 dict，processed_data 的结构类似：[{'type': 'wiki_summary', 'instruction': '请总结以下内容：', 'input': '原文内容', 'output': '总结内容'}, {}, ...]
            
        # 添加延时避免API限制
        # time.sleep(1)
        
        # 每处理100条数据保存一次
        if len(processed_data) % 10 == 0:
            save_interim_results(processed_data)
    
    # 最终保存
    save_final_results(processed_data)

def save_interim_results(data, filename="wiki_processed_interim.json"):
    """保存中间结果"""
    with open(filename, 'w', encoding='utf-8') as f: # w 是覆写，a 是追加
        json.dump(data, f, ensure_ascii=False, indent=2)

def save_final_results(data, filename="wiki_processed_final.json"):
    """保存最终结果"""
    # 按任务类型分类统计
    stats = {}
    for item in data:
        task_type = item['type']
        stats[task_type] = stats.get(task_type, 0) + 1 # 从 stats 中获取 task_type 的值，如果 task_type 不存在，则返回 0，如果存在，则返回 task_type 的值并加 1
    
    print("\n处理统计：")
    for task_type, count in stats.items():
        print(f"{task_type}: {count}条")
    
    # 保存数据
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"\n数据已保存至 {filename}")

In [24]:
if __name__ == "__main__":
    main()

处理维基百科数据:   0%|          | 3/51000 [00:14<61:56:04,  4.37s/it]

JSON解析错误: Invalid control character at: line 3 column 80 (char 112)


处理维基百科数据:   0%|          | 6/51000 [00:25<54:58:00,  3.88s/it]

JSON解析错误: Invalid control character at: line 3 column 55 (char 87)


处理维基百科数据:   0%|          | 9/51000 [00:35<44:59:09,  3.18s/it]

...

处理维基百科数据:   0%|          | 25/51000 [01:33<37:16:54,  2.63s/it]

JSON解析错误: Invalid control character at: line 3 column 62 (char 94)


处理维基百科数据:   0%|          | 29/51000 [01:47<42:25:49,  3.00s/it]

JSON解析错误: Invalid control character at: line 3 column 52 (char 84)


处理维基百科数据:   0%|          | 49/51000 [03:26<80:00:54,  5.65s/it] 